In [1]:
%load_ext Cython

In [2]:
%%cython
import numpy as np
#cimport numpy as np
import h5py
import cython

In [7]:
%%cython
import numpy as np
cimport numpy as np
from multiprocessing import Pool
cdef extern from "math.h":
    double atan2(double y, double x)
    
cdef double PI = np.pi

cdef class trajDiscretizationCython:
    cdef object centers
    cdef double innerMSMrad, entryRad, bathRad, bathRadOut, radialIncrementEntry, radialIncrementExit
    cdef int angularPartitions, entryRings, exitRings, radialPartitions, Ncenters, bathIndex
    def __init__(self, np.ndarray[double, ndim=2] centers):
        self.centers = centers
        self.innerMSMrad = 2.0
        self.entryRad = 2.6
        self.bathRad = 3.0
        self.bathRadOut = 4.0
        self.angularPartitions = 48
        self.entryRings = 6
        self.exitRings = 4
        self.radialPartitions = self.entryRings + self.exitRings
        self.radialIncrementEntry = (self.entryRad - self.innerMSMrad)/float(self.entryRings)
        self.radialIncrementExit = (self.bathRad - self.entryRad)/float(self.exitRings)
        self.Ncenters = len(self.centers)
        self.bathIndex = self.Ncenters + self.radialPartitions*self.angularPartitions
        #cpdef int self.lenCenters = len(self.centers)
        
    cdef int getAngularState(self, np.ndarray[double, ndim=1] coord):
        cdef double theta = atan2(coord[1], coord[0])
        cdef int angularState = int(theta*self.angularPartitions/(2*PI))
        return angularState
    
    cdef int getRing(self, double radius, double increment, double lowerBound):
        cdef int ring = 0
        cdef double upperBound = lowerBound
        while radius > upperBound:
            upperBound += increment
            ring += 1
        return ring - 1
    
    cdef int getRing2(self,double radius,double increment,double lowerBound):
        return int((radius-lowerBound)/increment)
    
    # Overwrite getState function in parent class
    #discretize trajectory: use two additional rings of states for entry and exit states
    #cluster volume should be on the same order of magnitute as for the internal states
    cdef int getState(self, np.ndarray[double, ndim=1] coord, int prevst):
        cdef double radius = np.linalg.norm(coord)
        cdef np.ndarray[double, ndim=2] cen2coord
        cdef np.ndarray[double, ndim=1] norm
        cdef int index
        # inner MSM
        if radius < self.innerMSMrad:
            for index in range(self.Ncenters):
                if (self.centers[index][0] - coord[0])**2 + (self.centers[index][1] - coord[1])**2 < 0.04:
                    return index
            return prevst
            '''
            cen2coord = self.centers - coord
            norm = np.linalg.norm(cen2coord, axis = 1)
            index = np.argmin(norm)
            if (norm[index] <= 0.2 ):
                return index
            # if no new state found, return previous state
            return prevst
        '''
        # Entry states
        elif radius < self.entryRad:
            angularState = self.getAngularState(coord)
            ringNumber = self.getRing(radius, self.radialIncrementEntry, self.innerMSMrad)
            return self.Ncenters + ringNumber * self.angularPartitions + angularState
        elif radius < self.bathRad:
            angularState = self.getAngularState(coord)
            ringNumber = self.getRing(radius, self.radialIncrementEntry, self.entryRad)
            return self.Ncenters + (ringNumber + self.entryRings) * self.angularPartitions + angularState
        elif radius > self.bathRad:
            angularState = self.getAngularState(coord)
            return self.Ncenters + self.radialPartitions*self.angularPartitions + angularState
        else:
            return prevst
    
    cpdef getdTrajs(self, list trajs):
        cdef np.ndarray[double, ndim=2] traj
        cdef list dTrajs = []
        cdef int k, checker,i 
        for traj in trajs:
            dTrajs.append(self.getdTraj(traj))
        return dTrajs
    
    cpdef list getdTraj(self, np.ndarray[double, ndim=2] traj):
        cdef int k, checker,i 
        cdef np.ndarray[long, ndim=1] dTraj
        # skip first elements, that might have udefined behavior.
        k = 0
        checker = self.getState(traj[0], -1)
        while checker < 0:
            k += 1
            checker = self.getState(traj[k], -1)
        dTraj = np.zeros(len(traj)-k, dtype=int)
        dTraj[0] = checker
        for i in range(1, len(traj)-k):
            dTraj[i] = self.getState(traj[i+k], dTraj[i-1])
        return dTraj

In [6]:
numfiles = 50
# Extract trajectories from file using asym2Dtraj class
fname = '../data/asym2D/2DasymTrajsLong_R4_RT1E7_dt001_SI10_'
minima = [[0.0,0.0], [1.0,0.0] , [1.1, 1.0], [-0.1,0.9], [-1.3,0.8], \
                       [-1.0,-0.2], [-0.6,-1.0], [0.9,-0.8], [0.2,-1.5]]
 # define allTrajs object
#alltrajs = allTrajs_inner([])
dimension = 2
trajs = []
for number in range(numfiles):
    f = h5py.File(fname + str(number) + '.h5', 'r')
    data = np.array(f['traj'], dtype=float)
    f.close()
    if not np.allclose(data[1:,0]-data[:-1,0], 10):
        print 'timestep not constant'
    traj = data[:,1:dimension+1]
    trajs.append(traj)
alltrajs = trajDiscretization.trajDiscretizationCython(np.array(minima), trajs)

In [7]:
%%time
dTrajs = alltrajs.parallelDtraj(4)

TypeError: expected string or Unicode object, NoneType found

In [111]:
%%cython -a
import numpy as np
cimport numpy as np
cdef extern from "math.h":
    double atan2(double y, double x)

def atan2C(double y, double x):
    return atan2(y, x)

def addSth(int a, int b):
    return a+b

In [120]:
print atan2C(0,-1)
print np.arctan2(0, -1)

3.14159265359
3.14159265359


In [30]:
%%cython

def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

In [29]:
geo_prog_cython(0.5, 5)

1.96875